In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.data.auto_mturk import get_drive_service, download_forms_urls, create_mturk_client
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from src.constants import TOKEN_PATH,CREDS_PATH, FORMS_URLS_PATH, AWS_KEYS_PATH
from src.utils import read_access_keys
import io
import shutil
import pandas as pd
from googleapiclient.http import MediaIoBaseDownload

In [4]:
# (1). Retrieve the urls from the app script

# retrieve gdrive service
service = get_drive_service()

# download the most recent forms_urls
download_forms_urls(FORMS_URLS_PATH,file_id,service)
forms_url = pd.read_csv(FORMS_URLS_PATH,sep=r"\s+",header=None,names=['url'],index_col=0)

NameError: name 'file_id' is not defined

In [65]:
forms_url = forms_url.url.to_dict()

In [32]:
# (2). Link to AWS Mturk


In [67]:
class Turker():
    def __init__(self,hittypeid,hitlayout,lifetimeinsec):
        """
        Args:
            hittypeid (str): hittypeid of the template to use
            hitlayout (str): hitlayout of the template to use
            lifetimeinsec (int): lifetime in seconds
        """
        # retrieval of the access keys
        aws_access_key_id,aws_secret_access_key = read_access_keys(AWS_KEYS_PATH)
        # creation of an self.client client
        self.client = create_mturk_client(aws_access_key_id,aws_secret_access_key)
        self.hittypeid = hittypeid
        self.hitlayout = hitlayout
        self.lifetimeinsec = lifetimeinsec

    def list_reviewable_hits(self):
        print(self.client.list_reviewable_hits()['HITs'])

    def __approve_all_assignments(self,hit_id):
        assignments = self.client.list_assignments_for_hit(HITId=hit_id,AssignmentStatuses=['Submitted'])
        assignments = assignments['Assignments']
        for ass in assignments:
            self.client.approve_assignment(AssignmentId=ass['AssignmentId'])

    def approve_all_hits(self):
        hit_ids = self.client.list_reviewable_hits()['HITs']
        for hit_id in hit_ids:
            self.__approve_all_assignments(hit_id)

    def delete_all_hits(self):
        hit_ids = self.client.list_reviewable_hits()['HITs']
        for hit_id in hit_ids:
            self.client.delete_hit(HITId=hit_id)

    def approve_delete_all_hits(self):
        self.approve_all_hits()
        self.delete_all_hits()

    def create_forms_hits(self,forms_url,hittypeid=None,hitlayout=None):
        """
        Args:
            forms_url(dict): mapping between forms index and their respective url        
        """
        hittypeid = hittypeid if hittypeid is None else self.hittypeid
        hitlayout = hitlayout if hitlayout is None else self.hitlayout
        for idx,url in forms_url.items():

            myhit = self.client.create_hit_with_hit_type(
                        HITTypeId=hittypeid,
                        HITLayoutId=hitlayout,
                        HITLayoutParameters = [{'Name':'url',
                                   'Value':url}],
                        LifetimeInSeconds = self.lifetimeinsec
        )

In [69]:
turk = Turker(hittypeid="3QA1H4GB4RCFG6O9NDVBQ4OP98C3MW",
             hitlayout="3QS25971A2UW6PKAU5WN9771C2JXNI",
             lifetimeinsec=600)

NameError: name 'create_mturk_client' is not defined

## Google urls file download

In [5]:
service = get_drive_service()

In [7]:
file_id = '11UjUJ9WfafdMEXZww9Q9GBnXsqX0dPWPdZGXGfFr_t4'

## Urls loading

In [10]:
forms_url = pd.read_csv(forms_path,sep=r"\s+",header=None,names=['url'],index_col=0)

## Personal Test

In [14]:
import boto3
from boto.mturk.connection import MTurkConnection
from boto.mturk.question import HTMLQuestion
from boto.mturk.layoutparam import LayoutParameter


## Personal test with boto3

In [48]:
# This will return your current MTurk balance if you are connected to Production.
# If you are connected to the Sandbox it will return $10,000.
print(client.get_account_balance()['AvailableBalance'])

10000.00


In [ ]:
myhit = client.create_hit_with_hit_type(
    HITTypeId="3QA1H4GB4RCFG6O9NDVBQ4OP98C3MW",
    HITLayoutId="3QS25971A2UW6PKAU5WN9771C2JXNI",
    HITLayoutParameters = [{'Name':'url',
                           'Value':'https://forms.gle/XKVGkivEhe2JwuEW7'}],
    LifetimeInSeconds = 3600,
    #AutoApprovalDelayInSeconds = 3600
)

In [57]:
print("A new HIT has been created. You can preview it here:")
if create_hits_in_production:
    print("https://worker.mturk.com/mturk/preview?groupId=" + myhit['HIT']['HITGroupId'])
else:
    print("https://workersandbox.mturk.com/mturk/preview?groupId=" + myhit['HIT']['HITGroupId'])
print("HITID = " + myhit['HIT']['HITId'] + " (Use to Get Results)")

A new HIT has been created. You can preview it here:
https://workersandbox.mturk.com/mturk/preview?groupId=3L2A3M5C18OOB5OIYYD5GARR4LJQG1
HITID = 3ZFRE2BDQ9Z76JMNSZMCL7GCCBMZXL (Use to Get Results)


## Retrieving Results

In [58]:
import boto3
import xmltodict

In [87]:
aws_access_key_id, aws_secret_access_key = read_access_keys("../creds/aws.txt")
mturk = boto3.client('mturk',
   aws_access_key_id = aws_access_key_id,
   aws_secret_access_key = aws_secret_access_key,
   region_name='us-east-1',
   endpoint_url = mturk_environment['endpoint']
)
# Use the hit_id previously created
hit_id = myhit['HIT']['HITId']
# We are only publishing this task to one Worker
# So we will get back an array with one item if it has been completed
worker_results = mturk.list_assignments_for_hit(HITId=hit_id, AssignmentStatuses=['Submitted'])

In [67]:
worker_results['Assignments']

{'NextToken': 'p1:cvOyx6c5NHPsyrOKIYEOevAs4Znua6m4+Qx5nFVhePlUvFipTfXIAWBAzYnivg==',
 'NumResults': 1,
 'Assignments': [{'AssignmentId': '3U5JL4WY5LUUP6VSX81OFUZJBTX4XI',
   'WorkerId': 'A29C1XYH77RQYM',
   'HITId': '3ZFRE2BDQ9Z76JMNSZMCL7GCCBMZXL',
   'AssignmentStatus': 'Submitted',
   'AutoApprovalTime': datetime.datetime(2020, 11, 23, 14, 39, 25, tzinfo=tzlocal()),
   'AcceptTime': datetime.datetime(2020, 11, 20, 14, 33, 5, tzinfo=tzlocal()),
   'SubmitTime': datetime.datetime(2020, 11, 20, 14, 39, 25, tzinfo=tzlocal()),
   'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>surveycode</QuestionIdentifier><FreeText>EMOJI378910</FreeText></Answer></QuestionFormAnswers>'}],
 'ResponseMetadata': {'RequestId': '5f907f92-38f2-4da9-9a15-54c081dc1cf6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5f907f92-38f2-4da9-9a1

In [79]:
display_answer(worker_results)

Worker's answer was:
For input field: surveycode
Submitted answer: EMOJI378910


In [81]:
answer

OrderedDict([('QuestionFormAnswers',
              OrderedDict([('@xmlns',
                            'http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd'),
                           ('Answer',
                            OrderedDict([('QuestionIdentifier', 'surveycode'),
                                         ('FreeText', 'EMOJI378910')]))]))])

In [78]:
def display_answer(worker_results):
    """
    Args:
        answer(dict): the xmldoc
    """
    if worker_results['NumResults'] > 0:
        for assignment in worker_results['Assignments']:
            xml_doc = xmltodict.parse(assignment['Answer'])

            print("Worker's answer was:")
            if type(xml_doc['QuestionFormAnswers']['Answer']) is list:
                # Multiple fields in HIT layout
                for answer_field in xml_doc['QuestionFormAnswers']['Answer']:
                    print("For input field: " + answer_field['QuestionIdentifier'])
                    print("Submitted answer: " + answer_field['FreeText'])
            else:
                # One field found in HIT layout
                print("For input field: " + xml_doc['QuestionFormAnswers']['Answer']['QuestionIdentifier'])
                print("Submitted answer: " + xml_doc['QuestionFormAnswers']['Answer']['FreeText'])
    else:
        print("No results ready yet")

In [80]:
worker_results

{'NextToken': 'p1:cvOyx6c5NHPsyrOKIYEOevAs4Znua6m4+Qx5nFVhePlUvFipTfXIAWBAzYnivg==',
 'NumResults': 1,
 'Assignments': [{'AssignmentId': '3U5JL4WY5LUUP6VSX81OFUZJBTX4XI',
   'WorkerId': 'A29C1XYH77RQYM',
   'HITId': '3ZFRE2BDQ9Z76JMNSZMCL7GCCBMZXL',
   'AssignmentStatus': 'Submitted',
   'AutoApprovalTime': datetime.datetime(2020, 11, 23, 14, 39, 25, tzinfo=tzlocal()),
   'AcceptTime': datetime.datetime(2020, 11, 20, 14, 33, 5, tzinfo=tzlocal()),
   'SubmitTime': datetime.datetime(2020, 11, 20, 14, 39, 25, tzinfo=tzlocal()),
   'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>surveycode</QuestionIdentifier><FreeText>EMOJI378910</FreeText></Answer></QuestionFormAnswers>'}],
 'ResponseMetadata': {'RequestId': '5f907f92-38f2-4da9-9a15-54c081dc1cf6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5f907f92-38f2-4da9-9a1

In [85]:
def answers_2_dataframe(worker_results):
    df = [{'WorkerID':assignment['WorkerId'],
           'HITId':assignment['HITId'],
           'Answer':xmltodict.parse(assignment['Answer'])} 
          for assignment in worker_results['Assignments']
           ]
    for answer in df:
        xml_doc = answer['Answer']
        
        if type(xml_doc['QuestionFormAnswers']['Answer']) is list:
            # Multiple fields in HIT layout
            for answer_field in xml_doc['QuestionFormAnswers']['Answer']:
                key = answer_field['QuestionIdentifier']
                answer_field['FreeText']
                answer[key] = value
        else:
            # One field found in HIT layout
            key = xml_doc['QuestionFormAnswers']['Answer']['QuestionIdentifier']
            value = xml_doc['QuestionFormAnswers']['Answer']['FreeText']
            answer[key] = value
        del answer['Answer']
    return pd.DataFrame(df)

In [86]:
answers_2_dataframe(worker_results)

,WorkerID,HITId,surveycode
0,A29C1XYH77RQYM,3ZFRE2BDQ9Z76JMNSZMCL7GCCBMZXL,EMOJI378910


# Dead Code

In [39]:
# Create your connection to MTurk
mtc = MTurkConnection(aws_access_key_id='',
                      aws_secret_access_key='',
                      host='mechanicalturk.sandboxdf.amazonaws.com')

In [25]:
url = LayoutParameter('url','https://forms.gle/XKVGkivEhe2JwuEW7')
params   = LayoutParameters([url])
response = mtc.create_hit(
  hit_layout    ="3QS25971A2UW6PKAU5WN9771C2JXNI",
  layout_params =params,
  hit_type      ="3QA1H4GB4RCFG6O9NDVBQ4OP98C3MW"
)

# The response included several fields that will be helpful later
hit_type_id = response[0].HITTypeId
hit_id = response[0].HITId
print("Your HIT has been created. You can see it at this link:")
print("https://workersandbox.mturk.com/mturk/preview?groupId={}".format(hit_type_id))
print("Your HIT ID is: {}".format(hit_id))

gaierror: [Errno -3] Temporary failure in name resolution